In [ ]:
# !pip install ctransformers[cuda]==0.2.27 \
#     gradio

In [ ]:
import gradio as gr
import time
from ctransformers import AutoModelForCausalLM


In [ ]:
def load_llm():
    llm = AutoModelForCausalLM.from_pretrained(
        # "../models/llama-2-13b-chat.Q8_0.gguf",
        "../models/llama-2-7b-chat.Q8_0.gguf",
        model_type='llama',
        max_new_tokens = 2048,
        repetition_penalty = 1.0,
        # temperature = 0.75,
        gpu_layers=150,
        context_length=4096
    )
    return llm

In [ ]:
def llm_function(message, chat_history):
    llm = load_llm()
    response = llm(
        message
    )
    output_texts = response
    return output_texts

In [ ]:
title = "Llama2 7B Quantized GGUF"

examples = [
    'Plan a 4 day trip to Japan, include the travel times.',
    'Plan a 5 day trip to Mumbai'
]

In [ ]:
gr.ChatInterface(
    fn = llm_function,
    title=title,
    examples=examples
).launch()